# RKNN Conversion Guide

----------------------------

### Before you start

Before you run the scripts or Python notebook from this project, it is recommended that you create a separate [Python virtual environment](https://docs.python.org/3/library/venv.html) so that the packages installed for the conversion process do not conflict with other packages you may already have installed.

### Preinstallation

This notebook requires the use of external Python scripts. Please run the installation script below to import these external scripts if you do not have them already.

In [ ]:
import subprocess

# Define scripts with URLs and inferred filenames
# DO NOT modify the filenames
scripts = [
    {
        "url": "https://raw.githubusercontent.com/boomermath/photonvision_rknn_fork/08117cc111c61264272188c8a1168b4c8e1bdb3b/scripts/rknn-convert-tool/create_onnx.py",
        "filename": "create_onnx.py"  # CREATE_ONNX_SCRIPT
    },
    {
        "url": "https://raw.githubusercontent.com/boomermath/photonvision_rknn_fork/08117cc111c61264272188c8a1168b4c8e1bdb3b/scripts/rknn-convert-tool/create_rknn.py",
        "filename": "create_rknn.py"  # CREATE_RKNN_SCRIPT
    }
]

# Download each script
for script in scripts:
    try:
        subprocess.run(["wget", script["url"], "-O", script["filename"]]).check_returncode()
        print(f"Successfully downloaded: {script['filename']}")
    except subprocess.CalledProcessError as e:
        print(f"Failed to download script from URL: {script['url']}")
        print(e)


#### *Numpy Fix* - Important for Google Colab Users

Google Colab comes with an incompatible version of Numpy installed. To fix this, please run the following cells below and **restart your session** when prompted.

In [1]:
%pip uninstall numpy -y
%pip install "numpy>=1.23.0,<2.0.0"

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Note: you may need to restart the kernel to use updated packages.
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Step 1: Convert to ONNX

To convert to ONNX, simply run the `create_onnx.py` script, providing the path to your model weights and specifying the model version, as shown below.

In [ ]:
# where version is either yolov5, yolov8, or yolov11, and model_path is the path to your weights file (.pt)
%run create_onnx.py --version yolov8 --model_path yolov8n.pt

### Step 2: Download RKNN API

You can either use `pip` below to automatically detect and install the correct RKNN API Python library for you, or install it manually.

#### Automatic installation

Please run `pip` below. If it does not work, refer to the instructions for manual installation.


In [ ]:
%pip install rknn-toolkit2

#### Manual Installation (If Automatic Installation Fails)
Visit the [RKNN Toolkit 2](https://github.com/airockchip/rknn-toolkit2) Github repository, then click on rknn-toolkit2, followed by packages.
If you are running an x86_64 CPU (e.g., most Intel and AMD processors), select that option; otherwise, choose arm64 for ARM-based computers (e.g., M-series Macs or Snapdragon processors). If you're unsure which CPU you're using, check your system settings for processor architecture information.

Once you've selected the correct CPU architecture, you'll see multiple packages. The file names will look something like:
`rknn_toolkit2-2.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl`.
The numbers after cp correspond to your Python version. For example, if you're using Python 3.10, look for a package with cp310 in the name. For Python 3.8, look for cp38; for Python 3.7, cp37, and so on.

Once you've found the correct package, click the "Raw" button to download the .whl file. Then, run the following command in your terminal, replacing rknn_toolkit2.whl with the actual path to the file you downloaded:

In [ ]:
%pip install rknn_toolkit2.whl

### Step 3: Convert to RKNN

To get started, run the `create_rknn.py` script, replacing the arguments with your own values. Refer to the table below for detailed information on each argument’s purpose and usage.

#### Overview of the `create_rknn.py` script

This script converts a YOLO ONNX model to RKNN format using a set of calibration images. It's designed to work with either:

- A flat directory of images (e.g. `train/images`), **or**
- A dataset directory containing a `data.yaml` file that defines `train`, `val`, and/or `test` folders.

##### Arguments

| Argument | Type | Description                                                                                                     |
|----------|------|-----------------------------------------------------------------------------------------------------------------|
| `--img_dir` (`-d`) | `str` (required) | Path to your image directory. This can either be a folder of images **or** a dataset folder with a `data.yaml`. |
| `--model_path` (`-m`) | `str` (required) | Path to your YOLO ONNX model, created in Step 1.                                                                |
| `--num_imgs` (`-ni`) | `int` (default: `300`) | Number of images to use for quantization calibration.                                                           |
| `--disable_quantize` (`-dq`) | `bool` (default: `False`) | Set to `True` to skip quantization entirely, not recommended for performance.                                   |
| `--rknn_output` (`-o`) | `str` (default: `out.rknn`) | File path where the final RKNN model should be saved.                                                           |
| `--img_dataset_txt` (`-ds`) | `str` (default: `imgs.txt`) | File path to store the list of images used during quantization.                                                 |
| `--verbose` (`-vb`) | `bool` (default: `False`) | Enable detailed logging from the RKNN API during conversion.                                                    |


##### *Note*

This script is designed for use with [PhotonVision](https://photonvision.org), and by default sets the target platform for RKNN conversion to `RK3588`, a chipset commonly found in many variants of the Orange Pi 5 series (e.g., Orange Pi 5, 5 Pro, 5 Plus, 5 Max, etc.). You may modify the `TARGET_PLATFORM` value in the `create_onnx.py` script to match your specific hardware or deployment requirements if necessary.

### Quantization Note

When performing quantization, it is critical to provide representative images of the objects or scenes you are trying to detect. These images are used to calibrate the model’s internal activations and greatly influence the final performance.

It is recommended to use 300–500 representative images that reflect the real-world input your model will encounter. As the old saying goes, it’s quality over quantity — having a diverse, relevant set matters more than simply having many images.

Quantization will cause some loss in model accuracy. However, if your calibration images are chosen wisely, this accuracy drop should be minimal and acceptable. If the sampled images are too uniform or unrelated, your quantized model's performance may worsen significantly.

The script will automatically sample representative images randomly from the provided dataset. While this usually works well, please verify that the dataset contains diverse and relevant examples of your target objects. As a reminder, the images used to quantize the model are stored in the text file specified by `--img_dataset_txt`.


### Optional: Download a dataset from Roboflow for quantization

If you do not already have a dataset or set of images containing the objects you want to detect, follow the steps below to download one from Roboflow Universe.

#### **Step 1: Search for a Dataset**

Go to [Roboflow Universe](https://universe.roboflow.com) and use the search bar to locate a dataset relevant to what you want to detect.
**Note:** The dataset must include the classes or object types you intend to detect.

#### **Step 2: Access the Dataset Tab**

After selecting a suitable project, navigate to the **Dataset** tab. Click the **"Download Dataset"** button. A prompt will appear with several options, including:

- Train a model with this dataset
- Train from a portion of this dataset
- Download dataset

Select **Download dataset**.

#### **Step 3: Choose Format and View Download Code**

- Under **Image and Annotation Format**, choose the version of YOLO you are using:
  - For **YOLOv5**, choose `YOLOv5 PyTorch`
  - For **YOLOv8**, choose `YOLOv8`
  - For **YOLOv11**, choose `YOLOv11`
- If multiple annotation formats are listed for your model, always select the one ending in **"PyTorch"**.

Then, under **Download Options**, click **"Show Download Code"** and continue.

In the resulting screen, you will see three tabs:
- **Jupyter**
- **Terminal**
- **Raw URL**

Select the **Terminal** tab and copy the provided command.

#### **Step 4: Paste and Run**

Paste the copied command into the notebook cell below and run it. This will download and extract the dataset into your environment, making it ready for use in the quantization process.

Make sure to prefix the command with "`!`" so it executes properly in this Jupyter Notebook environment.

In [ ]:
!curl -L "https://universe.roboflow.com/ds/FaF3HbDmF7?key=iMoJR25O9H" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

### RKNN Conversion Script

To quantize and convert the ONNX model to RKNN format, run the `create_rknn.py` script with the appropriate arguments. The `--model_path` argument should point to your exported ONNX model from Step 1, and `--img_dir` must reference a valid directory containing either a dataset or a set of images to be used for quantization.

##### *Note*

If you followed the Roboflow dataset download instructions from the previous section, the dataset will have been extracted to your **current working directory**. In that case, you can simply set `--img_dir` to "`.`" to reference the current directory.


In [ ]:
%run create_rknn.py --img_dir ./datasets --model_path weights.onnx

And that’s it! Your RKNN model file is now ready for deployment on an Orange Pi.